# Detekcja krawędzi

## Cel ćwiczenia

- Zapoznanie z metodami detekcji krawędzi:
    - Sobel, Prewitt, Roberts - przypomnienie,
    - Laplasjan z Gaussa (LoG – ang. Laplacian of Gaussian),
    - Canny.

Detekcja krawędzi przez wiele lat była podstawą algorytmów segmentacji.  
Krawędzie wykrywane są najczęściej z wykorzystaniem pierwszej (gradient) i drugiej (Laplasjan) pochodnej przestrzennej.  
Wykorzystanie obu metod zaprezentowane zostało w ćwiczeniu *Przetwarzanie wstępne. Filtracja kontekstowa*.

W niniejszym ćwiczeniu poznane detektory krawędzi zostaną porównane z bardziej zaawansowanymi: Laplasjan z funkcji Gaussa (LoG), Zero Crossing i Canny.

In [ ]:
import cv2
from matplotlib import pyplot as plt
import numpy as np
import math
import os
import requests

url = 'https://raw.githubusercontent.com/vision-agh/poc_sw/master/09_Canny/'

fileNames = ["dom.png"]
for fileName in fileNames:
    path = 'img/' + fileName
    if not os.path.exists(path):
        r = requests.get(url + fileName, allow_redirects=True)
        open(path, 'wb').write(r.content)

## Laplasjan z Gaussa (LoG)

### Teoria

Funkcja Gaussa:
\begin{equation}
h(r) = e^{\frac{-r^2}{2 \sigma^2}}
\end{equation}
gdzie:
- $r^2 = x^2 + y^2$
- $\sigma$ to odchylenie standardowe.

Działanie filtracji Gaussowskiej zostało przedstawione w ćwiczeniu "Przetwarzanie wstępne". W jej wyniku następuje rozmazanie obrazu.
Laplasjan tej funkcji dany jest wzorem:

\begin{equation}
\nabla^2 h(r) = \frac{r^2 - 2\sigma^2}{\sigma^4} e^{-\frac{r^2}{2\sigma^2}}
\end{equation}

Funkcję (z oczywistych powodów) nazywamy Laplasjan z Gaussa (LoG).
Ponieważ druga pochodna jest operacją liniową, konwolucja obrazu z $\nabla^2 h(r)$ daje taki sam efekt jak zastosowanie filtracji Gaussa na obrazie, a następnie obliczenie Laplasjanu z wyniku.
Lokalizacja krawędzi polega na znalezieniu miejsca, gdzie po filtracji LoG następuje zmiana znaku.

### Kroki

1. Wczytaj obraz *house.png*.
2. Wykonaj rozmycie Gaussowskie obrazu wejściowego.
W tym celu wykorzysaj funkcję `cv2.GaussianBlur(img, kSize, sigma)`.
Pierwszy argument jest obrazem wejśćiowym.
Drugi jest rozmiarem filtru (podanym w nawiasach okrągłych, np. *(3, 3)*).
Trzecim argumentem jest odchylenie standardowe. Wartość jest dobrana automatycznie, jeśli zosanie podana wartość `0` (będą równe rozmiarowi).
1. Oblicz laplasjan obrazu rozmytego.
W tym celu wykorzysaj funkcję `cv2.Laplacian(img, ddepth)`.
Pierszym argumentem jest obraz wejściowy.
Drugim argumentem jest typ danych wejściowych. Użyj `cv2.CV_32F`.
1. Wyznacz miejsca zmiany znaku.
Zaimplementuj funkcję `crossing(LoG, thr)`:
    - Najpierw stwórz tablicę, do której zostanie zapisany wynik.
    Jej rozmiar jest taki sam jak przetwarzanego obrazu.
    - Następnie wykonaj pętle po obrazie (bez ramki jednopikselowej).
    W każdej iteracji stwórz otoczenie o rozmiarze $3 \times 3$.
    Dla otoczenia oblicz wartość maksymalną i minimalną.
    - Jeśli wartości te mają przeciwne znaki, to do danego miejsca tablicy przypisz wartość:
        - jeśli piksel wejściowy > 0, to dodaj do niego wartość bezwzględną minimum.
        - jeśli piksel wejściowy < 0, to do jego wartości bezwzględnej dodaj maksimum.
    - Zmień zakres wykonanej tablicy do $<0, 255>$.
    - Wykonaj progowanie tablicy. Próg jest argumentem wejściowym.
    - Przeskaluj dane binarne do wartości `[0, 255]`.
    - Wykonaj konwersję do typu *uint8*.
    - Wykonaj rozmycie medianowe wyniku.
    Wykorzystaj funkcję `cv2.medianBlur(img, kSize)`.
    Pierwszym argumentem jest obraz wejśćiowy, a drugim rozmiar filtra.
    - Zwróć wyznaczoną tablicę.
1. Wyświetl obraz wynikowy.
2. Dobierz parametry (rozmiar filtru Gaussa, odchylenie standardowe, próg binaryzacji) tak, by widoczne były kontury domu, ale nie dachówki.

### Wykonanie

In [ ]:
def show(imgs, titles=None):
    if len(imgs) == 1:
        plt.figure(figsize=(10, 10))
        plt.imshow(imgs[0], cmap='gray')
        if titles is not None:
            plt.title(titles[0])
        plt.axis('off')
        plt.show()
        return
        
    _, axis = plt.subplots(1, len(imgs), figsize=(7*len(imgs), 7))
    for i, ax in enumerate(axis):
        ax.imshow(imgs[i], cmap='gray')
        if titles is not None:
            ax.set_title(titles[i])
        ax.axis('off')
    
    plt.show()

In [ ]:
def normalize(img):
    return (img - np.min(img)) / (np.max(img) - np.min(img))
    
def crossing(LoG, threshold, size=3):
    LoG_ = LoG.copy()
    size_ = int(size // 2)
    
    def window(x, y):
        return LoG[x-size_ : x+size_+1  ,  y-size_ : y+size_+1]
    
    def min_max(window):
        return np.min(window), np.max(window)
    
    for x in range(size_, LoG.shape[0] - size_):
        for y in range(size_, LoG.shape[1] - size_):
            min_, max_ = min_max(window(x, y))
            if min_ < 0 and max_ > 0:
                LoG_[x, y] += abs(min_) if LoG[x, y] > 0 else abs(max_)
    
    LoG_ = normalize(LoG_)

    LoG_thresholded = np.zeros_like(LoG_)
    LoG_thresholded[LoG_ < threshold] = 255
    # show([normalize(LoG), LoG_, LoG_thresholded])
    
    return cv2.medianBlur(LoG_thresholded.astype('uint8'), size)

In [ ]:
img = cv2.imread('img/dom.png', cv2.IMREAD_GRAYSCALE)
LoG = cv2.GaussianBlur(img, (9, 11), 0)

laplacian = cv2.Laplacian(LoG, cv2.CV_32F)

img_edges = crossing(laplacian, 0.475)

show([img, img_edges], ['oryginalny', 'krawędzie'])

## Algorytm Canny'ego

> Algorytm Canny'ego to często wykorzystywana metoda detekcji krawędzi.
> Zaproponowana została w 1986r. przez Johna F. Cannego.
> Przy jego projektowaniu założono trzy cele:
> - niska liczba błędów - algorytm powinien znajdywać wszystkie krawędzie oraz generować jak najmniej fałszywych detekcji,
> - punkty krawędziowe powinny być poprawnie lokalizowane - wykryte punkty powinny być jak najbardziej zbliżone do rzeczywistych,
> - krawędzie o szerokości 1 piksela - algorytm powinien zwrócić jeden punkt dla każdej rzeczywistej krawędzi.

### Kroki

1. W pierwszym kroku obraz przefiltruj dwuwymiarowym filtrem Gaussa.
2. Następnie oblicz gradient pionowy i poziomy ($g_x $ i $g_y$).
Jedną z metod jest filtracja Sobela.
3. Dalej oblicz amplitudę:
$M(x,y)  = \sqrt{g_x^2+g_y^2}$ oraz kąt:
$\alpha(x,y) = arctan(\frac{g_y}{g_x})$.
Do obliczenia kąta wykorzystaj funkcję `np.arctan2(x1, x2)`.
Wynik jest w radianach.
4. W kolejnym etapie wykonaj kwantyzację kątów gradientu.
Kąty od $-180^\circ$ do $180^\circ$ można podzielić na 8 przedziałów:
[$-22.5^\circ, 22.5^\circ$], [$22.5^\circ, 67.5^\circ$],
[$67.5^\circ, 112.5^\circ$], [$112.5^\circ, 157.5^\circ$],
[$157.5^\circ, -157.5^\circ$], [$-157.5^\circ, -112.5^\circ$],
[$-112.5^\circ, -67.5^\circ$], [$-67.5^\circ, -22.5^\circ$].
Przy czym należy rozpatrywać tylko 4 kierunki:
    - pionowy ($d_1$),
    - poziomy ($d_2$),
    - skośny lewy ($d_3$),
    - skośny prawy ($d_4$).
5. Dalej przeprowadź eliminację pikseli, które nie mają wartości maksymalnej (ang. *nonmaximal suppresion*).
Celem tej operacji jest redukcja szerokości krawędzi do rozmiaru 1 piksela.
Algorytm przebiega następująco.
W rozpatrywanym otoczeniu o rozmiarze $3 \times 3$:
    - określ do którego przedziału należy kierunek gradientu piksela centralnego ($d_1, d_2, d_3, d_4$).
    - przeanalizuje sąsiadów leżących na tym kierunku.
Jeśli choć jeden z nich ma amplitudę większą niż piksel centralny, to należy uznać, że nie jest lokalnym maksimum i do wyniku przypisać $g_N(x,y) = 0$.
W przeciwnym przypadku $g_N(x,y) = M(x,y)$.
Przez $g_N$ rozumiemy obraz detekcji lokalnych maksimów.
Zaimplementuj funkcję `nonmax`.
Pierwszym argementem jest macierz kierunków (po kwantyzacji).
Drugim argumentem jest macierz amplitudy.
6. Ostatnią operacją jest binaryzacja obrazu $g_N$.
Stosuje się tutaj tzw. binaryzację z histerezą.
Wykorzystuje się w niej dwa progi: $T_L$ i $T_H$, przy czym $T_L < T_H$.
Canny zaproponował, aby stosunek progu wyższego do niższego był jak 3 lub 2 do 1.
Rezultaty binaryzacji można opisać jako:<br>
$g_{NH}(x,y) = g_N(x,y) \geq T_H $<br>
$g_{NL}(x,y) = T_H > g_N(x,y) \geq T_L $<br>
Można powiedzieć, że na obrazie $g_{NH}$ są "pewne" krawędzie.
Natomiast na $g_{NL}$ "potencjalne".
Często krawędzie "pewne" nie są ciągłe.
Dlatego wykorzystuje się obraz $g_{NL}$ w następującej procedurze:
    - Stwórz stos zawierający wszystkie piksele zaznaczone na obrazie $g_{NH}$.
W tym celu wykorzystaj listę współrzędnych `[row, col]`.
Do pobrania elementu z początku służy metoda `list.pop()`.
Do dodania elementu na koniec listy służy metoda `list.append(new)`.
    - Stwórz obraz, który będzie zawierał informację czy dany piksel został już odwiedzony.
    - Stwórz obraz, któy zawierać będzie wynikowe krawędzie.
Jej rozmiar jest równy rozmiarowi obrazu.
    - Wykonaj pętlę, która będzie pobierać elementy z listy, dopóki ta nie będzie pusta.
W tym celu najlepiej sprawdzi się pętla `while`.
    - W każdej iteracji pobierz element ze stosu.
    - Sprawdź czy dany element został już odwiedzony.
    - Jeśli nie został, to:
        - Oznacz go jako odwiedzony,
        - Oznacz piksel jako krawędź w wyniku,
        - Sprawdź otoczenie piksela w obrazie $g_{NL}$,
        - Dodaj do stosu współrzędne otoczenia, które zawierają krawędź (potencjalną).
        Można to wykoanać np. pętlą po stworzonym otoczeniu.
7. Wyświetl obraz oryginalny, obraz $g_{NH}$ oraz obraz wynikowy.

Pomocnicze obrazy $g_{NH}$ i $g_{NL}$ zostały wprowadzone dla uproszczenia opisu.
Algorytm można zaimplementować wbardziej "zwarty" sposób.

Na podstawie powyższego opisu zaimplementuj algorytm Cannego.

### Wykonanie

In [ ]:
def canny(img, thresholds):
    size = 3
    size_ = int(size // 2)
    
    img = cv2.imread('img/dom.png', cv2.IMREAD_GRAYSCALE)
    img_blurred = cv2.GaussianBlur(img, (9, 11), 0)
    laplacian = cv2.Laplacian(img_blurred, cv2.CV_32F)
    LoG = crossing(laplacian, 0.475)
    
    # filtracja sobela
    sobel_x = cv2.Sobel(LoG, cv2.CV_32F, 1, 0, ksize=size)
    sobel_y = cv2.Sobel(LoG, cv2.CV_32F, 0, 1, ksize=size)
    
    def ditection(a):
        # kwantyzacja kontów:
        # d4 d3 d2       [ 5pi/8,  7pi/8] [ 3pi/8,  5pi/8] [  pi/8,  3pi/8]
        # d1    d1       [ 7pi/8, -7pi/8]                  [ -pi/8,   pi/8]
        # d2 d3 d4       [-7pi/8, -5pi/8] [-5pi/8, -3pi/8] [-3pi/8,  -pi/8]
        pi8 = np.pi / 8
        
        if     -pi8 <= a <=  pi8:
            return 1
        
        elif    pi8 <= a <=  3*pi8 or\
             -7*pi8 <= a <= -5*pi8:
            return 2
        
        elif  3*pi8 <= a <=  5*pi8 or\
             -5*pi8 <= a <= -3*pi8:
            return 3
        
        elif  5*pi8 <= a <=  7*pi8 or\
             -3*pi8 <= a <= -pi8:
            return 4

        return 1
    
    amplitude_s = np.sqrt(sobel_x**2 + sobel_y**2)
    alpha_s = np.arctan2(sobel_y, sobel_x)
    alpha = np.vectorize(ditection)(alpha_s)
    
    def window(x, y, img=LoG):
        return img[x-size_ : x+size_+1  ,  y-size_ : y+size_+1]
    
    def get_neighbours_in_direction(x, y):
        match alpha[x, y]:
            case 1:
                return LoG[x, y-1], LoG[x, y+1]
            case 2:
                return LoG[x-1, y+1], LoG[x+1, y-1]
            case 3:
                return LoG[x-1, y], LoG[x+1, y]
            case 4:
                return LoG[x-1, y-1], LoG[x+1, y+1]
    
    for x in range(size_, img.shape[0] - size_):
        for y in range(size_, img.shape[1] - size_):
            neighbors = get_neighbours_in_direction(x, y)
            if LoG[x, y] < max(neighbors):
                amplitude_s[x, y] = 0
                
    img_ = normalize(amplitude_s)
    img_NH = thresholds[1] <= img_
    img_NL = np.logical_and(thresholds[0] <= img_, img_ < thresholds[1])
    show([normalize(alpha), img_NH, img_NL], ['alpha', 'NH', 'NL'])
    
    img_edge = np.zeros_like(img_, dtype=bool)
    img_visited = np.zeros_like(img_, dtype=bool)
    fifo = [(x, y) for x, y in zip(*(img_NH > 0).nonzero())]
    while len(fifo) > 0:
        x, y = fifo.pop(0)
        if img_visited[x, y]:
            continue
        img_visited[x, y] = True
        img_edge[x, y] = True
        neighbors = window(x, y, img=img_NL)
        for x_, y_ in zip(*(neighbors > 0).nonzero()):
            fifo.append((x + x_ - size_, y + y_ - size_))
            
    return img_edge

In [ ]:
img = cv2.imread('img/dom.png', cv2.IMREAD_GRAYSCALE)
img_canny = canny(img, (0.1, 0.3))
show([img, img_canny], ['oryginalny', 'krawędzie'])

## Algorytm Canny'ego - OpenCV

### Kroki

1. Wykonaj dektekcję krawędzi metodą Canny'ego wykorzystując funkcję `cv2.Canny`.
    - Pierwszym argumentem funkcji jest obraz wejściowy.
    - Drugim argumentem jest mniejszy próg.
    - Trzecim argumentem jest większy próg.
    - Czwarty argument to tablica, do której wpisany zostanie wynik.
    Można zwrócić go przez wartość i podać wartość `None`.
    - Piąty argument to rozmiar operatora Sobela (w naszym przypadku 3).
    - Szósty argument to rodzaj używanej normy.
    0 oznacza normę $L_1$, 1 oznacza normę $L_2$. Użyj $L_2$.
2. Wynik wyświetl i porównaj z własną implementacją.

### Wykonanie

In [ ]:
img = cv2.imread('img/dom.png', cv2.IMREAD_GRAYSCALE)
img_canny = cv2.Canny(img, 70, 170, None, 3, True)
show([img, img_canny], ['oryginalny', 'krawędzie'])